In [1]:
from src.models.loader import DataModule
from src.models.heterolgn import HeteroLGN
from torch_geometric.utils import to_torch_coo_tensor
import torch

In [2]:
dm = DataModule()
x = next(iter(dm.val_dataloader()))
x

HeteroData(
  recipe={
    x=[196513, 9],
    id=[196513]
  },
  user={
    x=[175418, 0],
    id=[175418],
    input_id=[4096],
    batch_size=4096
  },
  (user, reviews, recipe)={
    edge_index=[2, 887502],
    edge_attr=[887502],
    edge_label=[113236],
    edge_label_index=[2, 113236]
  },
  (recipe, rev_reviews, user)={
    edge_index=[2, 871479],
    edge_attr=[871479]
  }
)

In [3]:
model = HeteroLGN(dm.num_users, dm.num_recipes)
model

HeteroLGN(
  (usr_embedding): Embedding(226570, 64)
  (rec_embedding): Embedding(231637, 64)
  (conv): HeteroLGConv()
  (ret_recall): RetrievalRecall()
  (ret_precision): RetrievalPrecision()
)

In [4]:
usr_out, rec_out = model(x.id_dict, x.edge_index_dict)

In [5]:
input_ids = x['user'].input_id
scores = usr_out[input_ids] @ rec_out.T
scores

tensor([[ 1.8263,  2.3983,  0.3323,  ..., -0.3137, -1.8601, -0.1606],
        [ 0.1502, -0.1459,  0.4423,  ...,  1.5803,  2.9280,  1.2659],
        [-0.8941, -0.7495, -1.3372,  ..., -0.8654, -1.2251, -0.0581],
        ...,
        [ 1.3590,  2.9683, -0.3041,  ..., -0.2137, -1.8921, -0.9454],
        [-0.5580,  0.3409, -0.3236,  ..., -0.1067,  1.1448, -0.0127],
        [ 0.8628, -0.8140, -0.4360,  ..., -2.5169, -2.5813, -1.2297]],
       grad_fn=<MmBackward0>)

In [9]:
scores.shape

torch.Size([4096, 196513])

In [10]:
k = 10
batch_size = scores.size(0)
top_scores = scores.topk(k, dim=-1).indices

preds = torch.sparse_coo_tensor(
    indices=torch.vstack([
        input_ids[:, None].expand_as(scores).flatten(),
        top_scores.flatten()
    ]),
    values=torch.ones(batch_size * k),
)
preds

: 

: 

In [107]:
batch_users, batch_recipes

(tensor(2966), tensor(3786))

AttributeError: 'torch.Size' object has no attribute 'sum'

In [113]:
target = to_torch_coo_tensor(x['reviews'].edge_label_index)
target

tensor(indices=tensor([[   0,    0,    0,  ..., 2964, 2965, 2966],
                       [ 866, 1734, 2666,  ...,  541, 2378, 2042]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(3787, 3787), nnz=4096, layout=torch.sparse_coo)